In [1]:
from bs4 import BeautifulSoup
import requests
import urllib.request, urllib.parse, urllib.error
import re
import ssl
import json
import calendar
import numpy as np
import pandas as pd
from datetime import datetime as dt

def get_related_article_list(keyword,num):
    url="https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&retmode=json&retmax=NUM&sort=relevance&term=KEYWORD"
    keyword = keyword.replace(" ","+")
    keyword = keyword.replace("(","%28")
    keyword = keyword.replace(")","%29")
    url = url.replace('NUM', str(num))
    url = url.replace('KEYWORD', keyword)
    webpage = urllib.request.urlopen(url).read()
    dict_page =json.loads(webpage)
    # print(dict_page)
    idlist = dict_page["esearchresult"]["idlist"]
    return idlist
def get_authors_name(i,article_dict,article_id):
    print("Articale S.No {} article_id {}".format(i,article_id))
    article_id = str(article_id)
    authors = article_dict['result'][article_id]["authors"]
    return authors

def get_authors_data(idlist,keyword,data):
#     data=[]
    for i in range(len(idlist)):
        url ="https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=pubmed&id=idlist&retmode=json"
        url = url.replace('idlist', idlist[i])
        articleResponse = urllib.request.urlopen(url).read()
        dict_article =json.loads(articleResponse)
        authors_list = get_authors_name(i,dict_article,idlist[i])
        for i in range(len(authors_list)):
            author_name=authors_list[i]["name"]
            author_name = author_name.replace(" ","+")
            url = "https://pubmed.ncbi.nlm.nih.gov/?term=KEYWORD"
            url = url.replace("KEYWORD",author_name)         
            author_all_Topic_res = requests.get(url)
            author_all_Topic_soup = BeautifulSoup(author_all_Topic_res.content, "html.parser")
            author_all_Topic_soup_res = author_all_Topic_soup.find('div', class_="results-amount")
            if(author_all_Topic_soup_res is not None):
                overall_number_of_articles = int(author_all_Topic_soup_res.find('span',class_="value").string.replace(",",""))
            else:
                overall_number_of_articles = 1
            if(overall_number_of_articles>=7 and overall_number_of_articles <=60):
                author_related_Topic = author_name +" AND "+keyword
                author_related_Topic = author_related_Topic.replace(" ","+")
                url = "https://pubmed.ncbi.nlm.nih.gov/?term=KEYWORD"
                url = url.replace("KEYWORD",author_related_Topic)         
                r = requests.get(url)
#                 print(url)
                soup = BeautifulSoup(r.content, "html.parser")
                number_of_articles = soup.find('span', class_="value")
                number_of_articles = 1 if (number_of_articles is None ) else int(number_of_articles.string.replace(",",""))
                if(number_of_articles>=3 and number_of_articles <=55):
                    articles_pub_year_list = soup.find_all('span',class_="docsum-journal-citation short-journal-citation")
                    article_year = list(map(lambda x:str(x).split('.')[1].strip(),articles_pub_year_list))
                    articles_pub_dates = []
                    article_ids=[]
                    article_years=[]
                    for i in range(len(article_year)):
                        if article_year[i] in ("2022","2021"):
                            article_date_string = str(articles_pub_year_list[i].find_previous_sibling("span",class_="docsum-journal-citation full-journal-citation")).split(";")[0].split(".")[1].strip()[:8]
                            if(len(article_date_string)==8):
                                article_pub_date = dt.strptime(article_date_string,'%Y %b')
                            else:
                                article_pub_date = dt.strptime(article_date_string,'%Y')
                            articles_pub_dates.append(article_pub_date.date())
                            article_ids.append(articles_pub_year_list[i].next_sibling.next_sibling.find('span',class_="docsum-pmid").string)
                            article_years.append(article_year[i])

                    authors_details=[]
                    if(('2021' or '2022') in article_year):
                        latest_date = max(articles_pub_dates,default=0)
                        latest_date_index = articles_pub_dates.index(latest_date)
                        latest_article_id = article_ids[latest_date_index]
                        authors_details.append(author_name)
                        authors_details.append(overall_number_of_articles)
                        authors_details.append(number_of_articles)
                        authors_details.append(latest_date)
                        authors_details.append(latest_article_id)
                        data.append(authors_details)
                        print("Name {} overall_count {} related_count {} latest_pub date {} article_Id {}".format(author_name,overall_number_of_articles,number_of_articles,latest_date,latest_article_id))
    return data
# Enter the keyWord required to search
keyword = '(pyrazole-Based inhibitors OR heteroaromatic scaffolds OR hydroxamate OR hydroxamic acid) AND (meprin OR metalloproteases OR metalloproteinases OR Synthesis OR Structure )';
#enter the number of articles like 100,200,300 count etc
num=100
# we can exclude already searched article Id's by slicing like below line
# idlist = :get_related_article_list(keyword,num)[100:] 
idlist = get_related_article_list(keyword,num)[23:25]
data=[]
data_res = get_authors_data(idlist,keyword,data)
df = pd.DataFrame(data_res,columns=['Author_name',"Overall_Published_Count",'Related no published articles',"Latest_published_date","Latest_published article_id"])
file_name = 'pyrazole.csv'
df=df.drop_duplicates()
df.to_csv(file_name)
print('DataFrame is written to Excel File successfully.')
df

Articale S.No 0 article_id 12798312
Articale S.No 1 article_id 17951059
DataFrame is written to Excel File successfully.


,Author_name,Overall_Published_Count,Related no published articles,Latest_published_date,Latest_published article_id


In [4]:
df1=df

In [5]:
df

,Author_name,Overall_Published_Count,Related no published articles,Latest_published_date,Latest_published article_id
0,Yasodha+R,10,4,2021-01-01,34047286
1,Sasi+A,41,3,2021-08-01,34539121
2,Janardhanan+A,14,3,2021-08-01,34539121
3,Wanga+VO,13,8,2022-08-01,35918646
4,Oulo+MA,11,8,2022-08-01,35918646
5,Mkala+EM,16,8,2022-08-01,35918646
6,Schwarzbach+AE,14,3,2021-03-01,33242585
7,Denoeud+F,41,3,2022-07-01,35015377
8,Alsos+IG,35,4,2022-07-01,35015377
9,Yaradua+SS,7,4,2021-06-01,34204211
